In [1]:
import sys
sys.path.append("../..")
import numpy as np
import focusadd
from focusadd.surface.Surface import Surface
from focusadd.coils.CoilSet import CoilSet
import numpy as np
import mayavi as maya
from mayavi import mlab
mlab.init_notebook()

Notebook initialized with x3d backend.


In [24]:
mlab.clf()
surface = Surface("../../focusadd/initFiles/axes/defaultAxis.txt", 128, 32, 1.0)
r = surface.get_r()
x = r[:,:,0]
y = r[:,:,1]
z = r[:,:,2]
s = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
coilSet = CoilSet(surface,input_file="validateFOCUS.hdf5")

Let's take a look at what FOCUSADD's coils look like. 

In [25]:
r_coils = coilSet.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.0,0.8))
p

Now let's overlay the FOCUSAD (FORTRAN) coils. First, we need to read in the data from coils.out.txt

In [23]:
with open("coils.out.txt", 'r') as f:
    _ = f.readline() #1
    NC, NF = f.readline().split(" ")
    NC = int(NC)
    NF = int(NF)
    _ = f.readline()
    NN, NB, NFR = f.readline().split(" ") #4
    NN = int(NN)
    NB = int(NB)
    NFR = int(NFR)
    _ = f.readline()
    _ = f.readline() #6
    fc = np.zeros((6,NC,NF+1))
    fr = np.zeros((2,NC,NFR))
    for c in range(NC):
        _ = f.readline() #coilnumber
        _ = f.readline() 
        _ = f.readline() #NR
        _ = f.readline() 
        _ = f.readline()
        xc = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        xs = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        yc = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        ys = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        zc = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        zs = np.asarray([float(txt) for txt in f.readline().split(" ") if txt != ''])
        _ = f.readline()
        _ = f.readline() 
        _ = f.readline()
        _ = f.readline()
        fc[:,c,:] = np.concatenate((xc[np.newaxis],yc[np.newaxis],zc[np.newaxis],xs[np.newaxis],ys[np.newaxis],zs[np.newaxis]),axis=0)

In [27]:
coilSet.set_params((fc,fr))


In [28]:
r_coils = coilSet.get_r()
for ic in range(r_coils.shape[0]):
    for n in range(r_coils.shape[2]):
        for b in range(r_coils.shape[3]):
            p = mlab.plot3d(r_coils[ic,:,n,b,0],r_coils[ic,:,n,b,1],r_coils[ic,:,n,b,2],tube_radius=0.004, line_width = 0.01, color=(0.0,0.8,0.0))
p

These are basically the same coils. Great. 